In [ ]:
!pip install -q datasets loralib sentencepiece
!pip uninstall transformers -y
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes
!pip install flask pyngrok -q

Found existing installation: transformers 4.27.0.dev0
Uninstalling transformers-4.27.0.dev0:
  Successfully uninstalled transformers-4.27.0.dev0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig
import textwrap



tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")

model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "samwit/alpaca7B-lora")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:

def alpaca_talk(text):
    prompt = f'''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{text} 

### Response:
'''

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.2,
    )
    print("Generating...")
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )
    ans = []
    for s in generation_output.sequences:
        # print(tokenizer.decode(s))
        ans.append(tokenizer.decode(s))
    return ans[0].split("### Response:")[-1]

In [ ]:
import requests
BASE_URL = "http://api.openweathermap.org/data/2.5/weather?"
API_KEY = "******"
CITY = "New York"


url = BASE_URL + "appid=" + API_KEY + "&q=" + CITY

#function to convert kelvin to celcius
def k2C(N):
  c=N-273
  return c

def Weatherget(city):
  url = BASE_URL + "appid=" + API_KEY + "&q=" + city
  response = requests. get(url).json()
# print(response)

  #getting the temp
  temp_kelvin = response['main']['temp']
  temp_celcius = k2C(temp_kelvin)
  max_temp = k2C(response['main']['temp_max'])
  min_temp = k2C(response['main']['temp_min'])

  #getting pressure and humidity
  pressure = response['main']['pressure']
  humidity= response['main']['humidity']

  #getting weather type
  weathertype = response['weather'][0]['main']
  weatherdesc = response['weather'][0]['description']

  #if rain occurs the give me the rain mm
  if weathertype == "rain":
    rainvol = response['rain']['1h']
  else:
    rainvol = 0
  

  #getting wind value
  windspeed = response['wind']['speed']
  wind_degree = response['wind']['deg']

  #cloud data
  cloud = response['clouds']['all']

  W= "No Warnings"

  #warnings
  if rainvol >= 100:
    W= "Flood incoming"
  elif rainvol >= 50:
    W= "The heavy rain fall may damage the crops"
  elif windspeed >= 33.33:
    W ="Cyclone incoming"
  elif max_temp >= 39:
    W = "There is a heate wave going in the region"
  elif min_temp <= 10:
    W= "There is a cold wave going in the region"

  A= f"City= {city}\n Temp(kelvin)= {temp_kelvin} \n Temp(Celcius) = {temp_celcius} \n Warnings = {W} \n Pressure = {pressure}hPa \n Humidity = {humidity}% \n Weather = {weathertype} \n Weather Description = {weatherdesc} \n Rain Volume = {rainvol}mm \n Wind Speed = {windspeed}m/s \n Wind Direction in degree = {wind_degree} \n Clouds = {cloud}%"
  return A
  # print(A)
# Weatherget(CITY)

In [ ]:
# Crops = "Rice"

# City = "New York"
# A= Weatherget(City)
# prompt = f"I am a farmer and I am cultivating {Crops} Give some suggestions in th following weather conditions {A}"
# Message= alpaca_talk(prompt)

In [ ]:
# !pip install dotenv
!pip install vonage

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# print (Message)

In [ ]:
# import os
# from os.path import join, dirname
# from dotenv import load_dotenv
import vonage
# dotenv_path = join(dirname(__file__), "../.env")
# load_dotenv(dotenv_path)

client = vonage.Client(key="233854f3", secret="ja80Tjf0NpZCsy56")
sms = vonage.Sms(client)# sms = vonage.Sms(client)

NEXMO_API_KEY = "****"
NEXMO_API_SECRET = "***"
TO_NUMBER = "919832103941"
NEXMO_BRAND_NAME = 'Green Data'

# import nexmo

# client = nexmo.Client(key=NEXMO_API_KEY, secret=NEXMO_API_SECRET)

responseData = sms.send_message(
    {
        "from": NEXMO_BRAND_NAME,
        "to": TO_NUMBER,
        "text": Message,
    }
)

if responseData["messages"][0]["status"] == "0":
    print("Message sent successfully.")
else:
    print(f"Message failed with error: {responseData['messages'][0]['error-text']}")

In [ ]:
!pip install twilio
# !pip install pyautogui 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.0 MB/s eta 0:00:00


In [ ]:
def report(name,phone,city,pin,crop):
  Z= Weatherget(CITY)
  Crops = "Rice"
  prompt = f"I am a farmer and I am cultivating {crop} Give some suggestions in th following weather conditions {Z}"
  Message= alpaca_talk(prompt)
  D = {"Name": name, "Weather": Z, "Response": Message}
  return D
# City = "New York"
# A= Weatherget(City)
print(report("Jeet",12333115,"Kolkata",7000105,"Rice"))

Generating...
{'Name': 'Jeet', 'Weather': 'City= New York\n Temp(kelvin)= 291.77 \n Temp(Celcius) = 18.769999999999982 \n Warnings = No Warnings \n Pressure = 1009hPa \n Humidity = 67% \n Weather = Clouds \n Weather Description = overcast clouds \n Rain Volume = 0mm \n Wind Speed = 1.79m/s \n Wind Direction in degree = 282 \n Clouds = 100%', 'Response': '\nSuggestions for growing rice under these conditions include using drip irrigation, planting early to avoid high temperatures, and choosing varieties suited to cooler climates such as short-grain or medium grains'}


In [ ]:
import os
import threading
from flask import Flask, request, render_template, jsonify, render_template_string
from pyngrok import ngrok, conf
import requests
import json



conf.get_default().auth_token = "2OJVuhSh4cupkKJmquQgWg1LZvm_248RTG6VbDzKhyed5ECdk"

os.environ["FLASK_ENV"] = "development"

app = Flask(__name__)
port = 5000

public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))


app.config["BASE_URL"] = public_url
app.config["FLASK_DEBUG"] = 1

@app.route('/')
def hello_world():
    return 'Hi, World!'

@app.route('/report/')
def report(n=None):
  name = request.args
  C = name["city"]
  Crops = name["crop"]
  Z= Weatherget(C)
  N= name["name"]
  Ph = name["ph"]
  # Crops = "Rice"
  prompt = f"I am a farmer and I am cultivating {Crops} Give some suggestions in th following weather conditions {Z}"
  Message= alpaca_talk(prompt)
  D = {"Name": name, "Weather": Z, "Response": Message}
  return D

threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()    


'FLASK_ENV' is deprecated and will not be used in Flask 2.3. Use 'FLASK_DEBUG' instead.


 * ngrok tunnel "https://8dca-34-143-181-58.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: on


'FLASK_ENV' is deprecated and will not be used in Flask 2.3. Use 'FLASK_DEBUG' instead.
'FLASK_ENV' is deprecated and will not be used in Flask 2.3. Use 'FLASK_DEBUG' instead.
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
!pkill ngrok
!pkill python